In [1]:
# Remove TF2 in colab
!pip uninstall -y tensorflow

In [2]:
# Download dependencies

!pip install -q -r requirements.txt
!apt-get install -qq libportaudio2

     |████████████████████████████████| 377.0MB 45kB/s 
     |████████████████████████████████| 686kB 38.9MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 245kB 33.5MB/s 
     |████████████████████████████████| 63.6MB 142kB/s 
     |████████████████████████████████| 3.2MB 41.8MB/s 
     |████████████████████████████████| 491kB 33.7MB/s 
     |████████████████████████████████| 204kB 43.1MB/s 
     |████████████████████████████████| 256kB 17.8MB/s 


In [3]:
# Download pretrained models

!gdown https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
!unzip pretrained.zip

Downloading...
From: https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
To: /content/pretrained.zip
384MB [00:02, 152MB/s]
Archive:  pretrained.zip
   creating: encoder/
   creating: encoder/saved_models/
  inflating: encoder/saved_models/pretrained.pt  
   creating: synthesizer/
   creating: synthesizer/saved_models/
   creating: synthesizer/saved_models/logs-pretrained/
   creating: synthesizer/saved_models/logs-pretrained/taco_pretrained/
 extracting: synthesizer/saved_models/logs-pretrained/taco_pretrained/checkpoint  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.data-00000-of-00001  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.index  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.meta  
   creating: vocoder/
   creating: vocoder/saved_models/
   creating: vocoder/saved_models/pretrained/
  inflating: vocoder/saved

In [4]:
# Get Helper funcs

!apt-get install -qq subversion
!svn checkout https://github.com/CorentinJ/Real-Time-Voice-Cloning/trunk/encoder
!svn checkout https://github.com/CorentinJ/Real-Time-Voice-Cloning/trunk/vocoder
!svn checkout https://github.com/CorentinJ/Real-Time-Voice-Cloning/trunk/synthesizer

Selecting previously unselected package libapr1:amd64.
(Reading database ... 144574 files and directories currently installed.)
Preparing to unpack .../libapr1_1.6.3-2_amd64.deb ...
Unpacking libapr1:amd64 (1.6.3-2) ...
Selecting previously unselected package libaprutil1:amd64.
Preparing to unpack .../libaprutil1_1.6.1-2_amd64.deb ...
Unpacking libaprutil1:amd64 (1.6.1-2) ...
Selecting previously unselected package libserf-1-1:amd64.
Preparing to unpack .../libserf-1-1_1.3.9-6_amd64.deb ...
Unpacking libserf-1-1:amd64 (1.3.9-6) ...
Selecting previously unselected package libsvn1:amd64.
Preparing to unpack .../libsvn1_1.9.7-4ubuntu1_amd64.deb ...
Unpacking libsvn1:amd64 (1.9.7-4ubuntu1) ...
Selecting previously unselected package subversion.
Preparing to unpack .../subversion_1.9.7-4ubuntu1_amd64.deb ...
Unpacking subversion (1.9.7-4ubuntu1) ...
Setting up libapr1:amd64 (1.6.3-2) ...
Setting up libaprutil1:amd64 (1.6.1-2) ...
Setting up libserf-1-1:amd64 (1.3.9-6) ...
Setting up libsvn1

In [0]:
import uuid
import librosa
import IPython
import warnings

import numpy as np
import ipywidgets as widgets

from pathlib import Path
from base64 import b64decode
from IPython.utils import io
from google.colab import output
from IPython.display import Audio
from IPython.display import Javascript
from encoder import inference as encoder
from vocoder import inference as vocoder
from synthesizer.inference import Synthesizer

warnings.filterwarnings('ignore')

In [0]:
# Don't actually need this for function calls, only for exemplar study

class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback) # refer to https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html for IPython events  

    template = """<button id="{callback_id}" style="cursor:pointer;background-color:#EEEEEE;border-color:#E0E0E0;padding:5px 15px;font-size:14px">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title = self._title, callback_id = callback_id)
    return html

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec = 3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb+') as f:
    f.write(b)
  return 'audio.wav'

In [7]:
encoder_weights = Path("encoder/saved_models/pretrained.pt")
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder/saved_models/pretrained/pretrained.pt


In [10]:
# Record in Ipython and display synthesized voice

def synth():
  text = "This is being said in my own voice.  The computer has learned to do an impression of me." # Text to be emulated by the speaker
  print("Now recording for 10 seconds, say what you will...")
  record(10)
  print("Audio recording complete")
  in_fpath = Path("audio.wav") # Audio file to be synthesized, can be changed to audio file of choice, refer synthesizer.py
  reprocessed_wav = encoder.preprocess_wav(in_fpath)
  original_wav, sampling_rate = librosa.load(in_fpath)
  preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
  embed = encoder.embed_utterance(preprocessed_wav)
  print("Synthesizing new audio...")
  with io.capture_output() as captured:
    specs = synthesizer.synthesize_spectrograms([text], [embed])
  generated_wav = vocoder.infer_waveform(specs[0])
  generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
  display(Audio(generated_wav, rate=synthesizer.sample_rate))
InvokeButton('Start recording', synth)

Now recording for 10 seconds, say what you will...


<IPython.core.display.Javascript object>

Audio recording complete
Synthesizing new audio...
{| ████████████████ 76000/76800 | Batch Size: 8 | Gen Rate: 6.6kHz | }